# Prerequisite


## Download nltk
This work makes havy use of the nltk-framework, the download can be startet with: '#nltk.download()'

In [ ]:
import nltk

nltk.download()

## Checking for GPU

*If you dont have a GPU in your maschine you can ignore these section*

For better performece we recomend using your GPU, if none is detectet consider adding it to your python-enviroment.

In [ ]:
import tensorflow as tf

gpu = tf.config.experimental.list_physical_devices('GPU')
cpu = tf.config.experimental.list_physical_devices('CPU')

print("There is", len(gpu), "detected GPU")
print("There is", len(cpu), "detected CPU")

# Loading the Data
We load the required data into to a pandas dataframes and join them by a common field.

for convinient use we transform all identifyers to lower case

In [ ]:
import pandas as pd

song_df = pd.read_csv('./song-lyrics/lyrics-data.csv')
author_df = pd.read_csv('./song-lyrics/artists-data.csv')

song_df = song_df.rename(columns={'ALink': 'Link'})

raw_df = song_df.merge(author_df, on='Link') 
raw_df.columns = raw_df.columns.str.lower()

print("There are", len(raw_df), "datasets loaded")

# Remove unneedet data

we removed all samples which arnt english or are duplicates

In [ ]:
import pandas as pd

columns_to_drop = ['sname','slink','link', 'popularity', 'genres', 'songs', 'link', 'idiom']

filterd_df = raw_df[raw_df.idiom.eq('ENGLISH')]
filterd_df = filterd_df.drop_duplicates(subset='slink', keep='first') 
filterd_df = filterd_df.drop(columns_to_drop, axis=1)

# General preprocessing

for further procssesing the lyrics get tokenized, lemmatized and cleand

In [ ]:
import pandas as pd

transformed_df = filterd_df

## Lowercase

In [ ]:
import pandas as pd

transformed_df['lyric'] = transformed_df['lyric'].str.lower()

## Tokenize

In [ ]:
import nltk

def lyric_to_verse(lyric):
    return nltk.sent_tokenize(lyric,language='english')

def verse_to_word(verse):
    return nltk.word_tokenize(verse, language='english')

transformed_df['lyric'] = transformed_df['lyric'].transform(lambda x: lyric_to_verse(x)) 
transformed_df['lyric'] = transformed_df['lyric'].transform(lambda x: [verse_to_word(verse) for verse in x]) 

## Lemmatiz

now we lemmatize our tokens, this may take a wile

In [ ]:
import nltk
from nltk.corpus import wordnet


#by http://textmining.wp.hs-hannover.de/Preprocessing.html
def pos_tagger(sentences):
    return [nltk.pos_tag(sent) for sent in sentences]

def wntag(pttag):
    if pttag in ['JJ', 'JJR', 'JJS']:
        return wordnet.ADJ
    elif pttag in ['NN', 'NNS', 'NNP', 'NNPS']:
        return wordnet.NOUN
    elif pttag in ['RB', 'RBR', 'RBS']:
        return wordnet.ADV
    elif pttag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
        return wordnet.VERB
    return None

def lemmatize(lemmatizer,word,pos):
    if pos == None:
        return word
    else:
        return lemmatizer.lemmatize(word,pos)
    
def lem(sentences):
    transformed = []
    for sentence in sentences:
        transformed.append([lemmatize(lemmatizer,word,wntag(pos)) for (word,pos) in sentence])
    return transformed 

lemmatizer = nltk.WordNetLemmatizer()

transformed_df['lyric'] = transformed_df['lyric'].transform(lambda x: pos_tagger(x)) 
transformed_df['lyric'] = transformed_df['lyric'].transform(lambda x: lem(x)) 

## Cleanup

we remove any artefacts which surviced lematisation like empty words, single letter words and all non alphabetic characters 

In [ ]:
import re

def clean_non_alphabetic_characters(sentences):
    cleaned = []
    for sentence in sentences:
        cleaned.append(list(map(lambda x: re.sub("[\W_]", '', x), sentence)))
    return cleaned

def clean_empty_words(sentences):
    cleaned = []
    for sentence in sentences:
        cleaned.append(list(filter(lambda x: x != '' and x != "", sentence)))
    return cleaned

def clean_single_letters(sentences):
    cleaned = []
    for sentence in sentences:
        cleaned.append(list(filter(lambda x: len(x) != 1, sentence)))
    return cleaned

def clean_empty_sentences(sentences):
    return list(filter(lambda x : len(x) != 0, sentences))

transformed_df['lyric'] = transformed_df['lyric'].transform(lambda x: clean_non_alphabetic_characters(x)) 
transformed_df['lyric'] = transformed_df['lyric'].transform(lambda x: clean_empty_words(x))  
transformed_df['lyric'] = transformed_df['lyric'].transform(lambda x: clean_single_letters(x)) 
transformed_df['lyric'] = transformed_df['lyric'].transform(lambda x: clean_empty_sentences(x)) 

# Optional preprocessing


## Remove to long Lyrics

checken ob lieder die länger wie 1000 oder 1500 wirklich save sind, am besten erstmal händisch nachschauen

In [ ]:
#todo

## Remove additional Genres

alle generes rauswerfen die für weniger als x lieder haben 

In [ ]:
#todo

# Store preproccesd data

## Save to disk

In [ ]:
transformed_df.to_csv (r'./export_dataframe.csv', index = False, header=True)
transformed_df.head()

## Load form disk

In [ ]:
import pandas as pd

transformed_df = pd.read_csv(r'./export_dataframe.csv', converters={'lyric': eval})
transformed_df.head()

## Helper

kleiner helfer um firsche dfs zu bekommen falls man destruktiv daten bearbeitet

In [ ]:
from helper import flatten
import pandas as pd
import pickle

restored_df = pd.read_csv(r'./export_dataframe.csv', converters={'lyric': eval})

def get_fresh_copy(frac=1):
    copy_df = pickle.loads(pickle.dumps(restored_df.sample(frac=frac, random_state=1)))
    return copy_df

def get_fresh_flatted_copy(frac=1):
    copy_df = get_fresh_copy(frac)
    copy_df['lyric'] = copy_df['lyric'].transform(lambda x: flatten(x))
    return copy_df
    

# Look at the data

## GenreDistribution

first we will look at the distribution of generes in our dataset

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))
sns.countplot(data=get_fresh_copy())

## FreqencyDistribution

freqency distribution of the most common words in our dataset

to see if removing stopwords is a good idea we have a look ate the destribuition of stopwords over the difertent genres

In [ ]:
#todo refactor 

import itertools
import seaborn as sns
import nltk
import matplotlib.pyplot as plt
import pandas as pd
        
flatted_df = get_fresh_flatted_copy(1)
genres = flatted_df.genre.unique()

df_container = {}
fd_container = {}

total_word_count = {}

merged_fd_df = pd.DataFrame(data = nltk.FreqDist(itertools.chain.from_iterable(flatted_df['lyric'])).items(), columns=['words', 'frequency'])
merged_fd_df = merged_fd_df.sort_values("frequency", ascending=False)

#in viele kleine loops zerhacken und erklären was passiert
#und vielleicht noch den duchschnitt hinzufügen, allso nicht nach genre gefilterd
for genre in genres:
    df_container[genre] = flatted_df[flatted_df.genre.eq(genre)]
    fd_container[genre] = nltk.FreqDist(itertools.chain.from_iterable(df_container[genre]['lyric']))
    total_word_count[genre] = sum(fd_container[genre].values())
    fd_container[genre] = pd.DataFrame(fd_container[genre].items(), columns=['words', genre])
    fd_container[genre][genre] = fd_container[genre][genre].transform(lambda x: x/total_word_count[genre]) 
    merged_fd_df = merged_fd_df.merge(fd_container[genre], on='words')

    
merged_fd_df = merged_fd_df.drop(['frequency'], axis=1)
merged_fd_df = merged_fd_df.set_index('words')

plt.figure(figsize=(20, 10))
plt.xticks(rotation=45)    
sns.lineplot(data=merged_fd_df.head(50))

## Lyricslength Distribution by genre(word)

we look if some generes as more likly to have shorter or longer lyrics

In [ ]:
def freqenzyDistribution(data, level='lyric'):
    data[level] = data[level].transform(lambda x: len(x))

    genres = data.genre.unique()

    df_container = {}
    mean_container = {}
    std_container = {}
    counts_container = {}

    for genre in genres:
        df_container[genre] = pickle.loads(pickle.dumps(data[data.genre.eq(genre)]))

    for genre in genres:
        mean_container[genre] = df_container[genre][level].values.mean()

    for genre in genres:
        std_container[genre] = df_container[genre][level].values.std()

    for genre in genres:
        counts_container[genre] = df_container[genre][level].value_counts(normalize=True)

    for genre in genres:
        counts_container[genre] = pd.DataFrame.from_dict(counts_container[genre])
        counts_container[genre]['genre'] = genre
        counts_container[genre] = counts_container[genre].reset_index()
        counts_container[genre] = counts_container[genre].rename(columns={level: 'count'})
        counts_container[genre] = counts_container[genre].rename(columns={'index': 'length'})

    return pd.concat(counts_container.values())

In [ ]:
import itertools
import seaborn as sns
import nltk
import matplotlib.pyplot as plt
import pandas as pd
    
        
flatted_df = get_fresh_flatted_copy(1)
result_df = freqenzyDistribution(flatted_df)

plt.figure(figsize=(20, 10))
plt.xticks(rotation=45)
sns.scatterplot(data=result_df[result_df['count']>= 0.001], x="length", y="count", hue="genre")

## Lyricslength Distribution by genre(vers)

we look if some generes as more likly to have shorter or longer lyrics

In [ ]:
import itertools
import seaborn as sns
import nltk
import matplotlib.pyplot as plt
import pandas as pd
from helper import melt

flatted_df = get_fresh_copy(1)
flatted_df = pd.DataFrame(melt(flatted_df))

result_df = freqenzyDistribution(flatted_df, 'verse')

plt.figure(figsize=(20, 10))
plt.xticks(rotation=45)
sns.lineplot(data=result_df[result_df['count']>= 0.001], x="length", y="count", hue="genre")

## Lyricslength Distribution by genre(lyric)

we look if some generes as more likly to have shorter or longer lyrics

In [ ]:
import itertools
import seaborn as sns
import nltk
import matplotlib.pyplot as plt
import pandas as pd

flatted_df = get_fresh_copy(1)
result_df = freqenzyDistribution(flatted_df)

plt.figure(figsize=(20, 10))
plt.xticks(rotation=45)
sns.lineplot(data=result_df[result_df['count']>= 0.001], x="length", y="count", hue="genre")

# Learning

In [ ]:
#todo recfactor

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
import keras

preprocessed_df = get_fresh_flatted_copy()

x_train, x_test, y_train, y_test = train_test_split(preprocessed_df['lyric'], preprocessed_df['genre'], test_size=0.20)
tokenizer = Tokenizer(num_words=10000)

le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)

num_classes =  len(transformed_df['genre'].unique())
tokenizer.fit_on_texts(x_train)

print(num_classes)

x_train = tokenizer.texts_to_matrix(x_train, mode='tfidf')
x_test = tokenizer.texts_to_matrix(x_test, mode='tfidf')

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation


model = Sequential()
model.add(Dense(4096, input_shape=(10000, ), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='linear'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 32
epochs = 1

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.1)
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
